<a href="https://colab.research.google.com/github/deepinmachine/classification-of-customers/blob/main/Whether_a_person_will_by_health_insurance_or_not.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Problem statement***

Our client is an Insurance company that has provided Health Insurance to its customers now they need your help in building a model to predict whether the policyholders (customers) from past year will also be interested in Vehicle Insurance provided by the company. An insurance policy is an arrangement by which a company undertakes to provide a guarantee of compensation for specified loss, damage, illness, or death in return for the payment of a specified premium. A premium is a sum of money that the customer needs to pay regularly to an insurance company for this guarantee. For example, you may pay a premium of Rs. 5000 each year for a health insurance cover of Rs. 200,000/- so that if, God forbid, you fall ill and need to be hospitalised in that year, the insurance provider company will bear the cost of hospitalisation etc. for upto Rs. 200,000. Now if you are wondering how can company bear such high hospitalisation cost when it charges a premium of only Rs. 5000/-, that is where the concept of probabilities comes in picture. For example, like you, there may be 100 customers who would be paying a premium of Rs. 5000 every year, but only a few of them (say 2-3) would get hospitalised that year and not everyone. This way everyone shares the risk of everyone else.

Just like medical insurance, there is vehicle insurance where every year customer needs to pay a premium of certain amount to insurance provider company so that in case of unfortunate accident by the vehicle, the insurance provider company will provide a compensation (called ‘sum assured’) to the customer.

Building a model to predict whether a customer would be interested in Vehicle Insurance is extremely helpful for the company because it can then accordingly plan its communication strategy to reach out to those customers and optimise its business model and revenue.

Now, in order to predict, whether the customer would be interested in Vehicle insurance, you have information about demographics (gender, age, region code type), Vehicles (Vehicle Age, Damage), Policy (Premium, sourcing channel) etc

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
path='/content/TRAIN-HEALTH INSURANCE CROSS SELL PREDICTION.csv'
df=pd.read_csv(path)

In [4]:
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44.0,1.0,28.0,0.0,> 2 Years,Yes,40454.0,26.0,217.0,1.0
1,2,Male,76.0,1.0,3.0,0.0,1-2 Year,No,33536.0,26.0,183.0,0.0
2,3,Male,47.0,1.0,28.0,0.0,> 2 Years,Yes,38294.0,26.0,27.0,1.0
3,4,Male,21.0,1.0,11.0,1.0,< 1 Year,No,28619.0,152.0,203.0,0.0
4,5,Female,29.0,1.0,41.0,1.0,< 1 Year,No,27496.0,152.0,39.0,0.0


**Explaining the data**


1.   Id -Uniqe id of each customer
2.   Gender - Male or Female
3.   Age - age of customer
4. Driving license- whether customer have DL or not
5. Region- particular code for rach region
6. Previously_Insured- yes or no
7. Vehicle_Age -Age of vehicle
8. Vehicle_Damage- Past damages present or not
9. Annual_Premium- The amount customer needs to pay as premium
10. Policy_Sales_Channel - Channel used to outreaching the customer
11. Vintage- Number of days customer has been associated with the company
12. Response- Customer is interested or not




This is a  Supervised classification problem where we have to do binary classification whether a customer will be interestd in buy a insurance or not.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37960 entries, 0 to 37959
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    37960 non-null  int64  
 1   Gender                37959 non-null  object 
 2   Age                   37959 non-null  float64
 3   Driving_License       37959 non-null  float64
 4   Region_Code           37959 non-null  float64
 5   Previously_Insured    37959 non-null  float64
 6   Vehicle_Age           37959 non-null  object 
 7   Vehicle_Damage        37959 non-null  object 
 8   Annual_Premium        37959 non-null  float64
 9   Policy_Sales_Channel  37959 non-null  float64
 10  Vintage               37959 non-null  float64
 11  Response              37959 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 3.5+ MB


There are no null values present.

Vehicle age must be a float value.

Vehicle damage is binary value.

In [6]:
df.describe()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,37960.000000,37959.000000,37959.000000,37959.000000,37959.000000,37959.000000,37959.000000,37959.000000,37959.000000
mean,18979.500079,38.773440,0.997892,26.453990,0.456993,30563.669538,112.569430,154.090835,0.121763
std,10958.252309,15.540311,0.045860,13.168572,0.498154,17051.398032,53.935742,83.574081,0.327016
min,1.000000,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,9489.750000,25.000000,1.000000,15.000000,0.000000,24384.500000,30.000000,82.000000,0.000000
50%,18979.500000,36.000000,1.000000,28.000000,0.000000,31746.000000,151.000000,154.000000,0.000000
75%,28469.250000,49.000000,1.000000,35.000000,1.000000,39518.500000,152.000000,227.000000,0.000000
max,37959.000000,84.000000,1.000000,52.000000,1.000000,508073.000000,163.000000,299.000000,1.000000


Annual premium seems to have some outliers.

Response mean very low this signify that numbers of 0 are high.

Almost everyone in our data have a driving license

In [15]:
df['Response'].value_counts()

0.0    33337
1.0     4622
Name: Response, dtype: int64

This is not a balanced data percentage of 1 is very low compared to zero.

So, The use Ensemble techiniques like Random forest, Gradients Boosting can help us . and abount selecetion of algorithm we could use Decision trees and Random forest.

I can also use RandomSampler in case of logistic regression as it the algorithm that i want to use currently.